# Interacting with Recommenders, Campaigns and Filters <a class="anchor" id="top"></a>

Now that Unicorn Post has trained models for 2 different use cases (User Personalization and Personalized Reranking), we need to integrate them into our application. Amazon Personalize can make recommendations available via an Application Programming Interface (API). In addition, Amazon Personalize includes features that allow you to easily integrate into applications and provide benefits like real time vending of recommendations based on recent application activity.

In this notebook, you will interact with campaigns and filters you created earlier in Amazon Personalize.

1. [Introduction](#intro)
1. [Interact with Recommenders](#interact-recommenders)
1. [Interact with Campaigns](#interact-campaigns)
1. [Filters](#filters)
1. [Create Filters](#create-filters)
1. [Using Filters](#using-filters)
1. [Real-time Events](#real-time)
1. [Wrap Up](#wrapup)

To run this notebook, you need to have run the previous notebooks, [`01_Data_Layer.ipynb`](01_Data_Layer.ipynb), and [`02_Training_Layer.ipynb`](02_Training_Layer.ipynb), where you created a dataset and imported interaction, item, and user metadata data into Amazon Personalize, created recommenders, solutions, and campaigns.

## Introduction <a class="anchor" id="intro"></a>
[Back to top](#top)

At this point, you should have two deployed Campaign. Once they are active, there are resources for querying the recommendations, and helper functions to digest the output into something more human-readable. 

In this Notebook we will interact with the Campaigns and get recommendations. 

We will interact with filters and send live data to Amazon Personalize to see the effect of real-time interactions on recommendations.

The following diagram shows the resources that we will create in this section. The part we are building in this notebook highlighted in blue with a dashed outline.

![Workflow](Images/03_Inference_Layer_Resources.jpg)

To get started, once again, we need to import libraries, load values from previous notebooks, and load the SDK.

In [2]:
import time
from time import sleep
import json
from datetime import datetime
import uuid
import random
import boto3
import pandas as pd

In [3]:
#retrieves previously stored variables 
%store -r 

In [4]:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# Establish a connection to Personalize's event streaming
personalize_events = boto3.client(service_name='personalize-events')

## Interact with Campaigns <a class="anchor" id="interact-recommenders"></a>
[Back to top](#top)

Now that the models have been trained, lets have a look at the recommendations we can get for our users!

### User Personalization Model

"User Personalization " requires a user as input, and it will return the items it thinks the customer is most likely to interact with next.

The cells below will handle getting recommendations from the "User Personalization Model" and rendering the results. Let's see what the recommendations are for a user.

We will be using the `campaignArn`, the `userId`, as well as the number or results we want, `numResults`.

### Select a User

We'll just pick a random user for simplicity. Feel free to change the `user_id` below and execute the following cells with a different user to get a sense for how the recommendations change.

#### Sample User ID's
 -8845298781299428018
 -1032019229384696495
 -1130272294246983140
 344280948527967603
 -445337111692715325

In [5]:
sample_user = str(-8845298781299428018)

In [6]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = workshop_userpersonalization_campaign_arn,
    userId = sample_user,
    numResults = 5
)

In [7]:
print(get_recommendations_response)

{'ResponseMetadata': {'RequestId': '97091ab0-1154-4011-84e5-2fb8f3d2eb82', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Tue, 20 Feb 2024 14:04:45 GMT', 'content-type': 'application/json', 'content-length': '614', 'connection': 'keep-alive', 'x-amzn-requestid': '97091ab0-1154-4011-84e5-2fb8f3d2eb82'}, 'RetryAttempts': 0}, 'itemList': [{'itemId': '3566197569262766169', 'score': 0.6605853}, {'itemId': '-746073086109727488', 'score': 0.0336421}, {'itemId': '-4029704725707465084', 'score': 0.0331361}, {'itemId': '-8900113512825364282', 'score': 0.0201865}, {'itemId': '-1415040208471067980', 'score': 0.0092185}], 'recommendationId': 'RID-b2-4c22-8b65-922067b3efdb-CID-c97f13'}


A little hard to read - lets make a dataframe

In [8]:
recommendations_df = pd.DataFrame.from_records(get_recommendations_response['itemList'])

In [9]:
recommendations_df

,itemId,score
0,3566197569262766169,0.660585
1,-746073086109727488,0.033642
2,-4029704725707465084,0.033136
3,-8900113512825364282,0.020186
4,-1415040208471067980,0.009218


Lets lookup what sort of articles are being recommended to this user

In [10]:
articles_mlfeatures['item_id'] = articles_mlfeatures['item_id'].astype('str')

In [12]:
recommendations_df = recommendations_df.merge(articles_mlfeatures, how='left', left_on='itemId', right_on='item_id')

In [13]:
recommendations_df

,itemId,score,creation_timestamp,item_id,lang,article_genre,training_text
0,3566197569262766169,0.660585,1486520793,3566197569262766169,en,cloud provider news,Google extends Gmail API for more granular ema...
1,-746073086109727488,0.033642,1486517733,-746073086109727488,en,non tech,5 key questions marketing executives should as...
2,-4029704725707465084,0.033136,1487246811,-4029704725707465084,en,non tech,Career coach advises making a mind map to desi...
3,-8900113512825364282,0.020186,1486999803,-8900113512825364282,en,non tech,Report finds banks lag in customer experience ...
4,-1415040208471067980,0.009218,1482409156,-1415040208471067980,en,cloud provider news,Google Cloud Platform releases icon library fo...


In [14]:
for i, row in recommendations_df.iterrows():
    print(row['training_text'])
    print("===================================")

Google extends Gmail API for more granular email settings management Google updated the Gmail API with new endpoints to manage filters, aliases, forwarding, signatures, vacation responders, and other granular email settings. This replaces the deprecated Email Settings API. Google has extended the Gmail API with new endpoints for managing email settings like filters, forwarding addresses, IMAP/POP settings, sendas aliases, signatures, and vacation responders. Developers can now retrieve and update signatures for sendas aliases, configure forwarding to external addresses, configure sendas aliases through external providers, use HTML in vacation messages, and manipulate settings for gmail.com accounts. More settings features like mailbox delegate support will be added over time. Most settings endpoints work for any Google Apps or Gmail account, but sensitive operations like modifying aliases or forwarding are restricted to service accounts with domainwide authority. The existing Email Set

What has this user viewed previously

In [15]:
viewed_interactions = interaction_data[interaction_data['user_id'].astype(str) == sample_user].sort_values('timestamp', ascending=False)

In [16]:
viewed_interactions

,timestamp,event_type,item_id,user_id,session_id,user_device_type
69083,1487360168,FOLLOW,3566197569262766169,-8845298781299428018,808768479044973017,NonMobile
69085,1487360167,COMMENT CREATED,3566197569262766169,-8845298781299428018,808768479044973017,NonMobile
69082,1487359849,VIEW,3566197569262766169,-8845298781299428018,808768479044973017,NonMobile
69722,1487069095,VIEW,-8900113512825364282,-8845298781299428018,8663979798581613597,NonMobile
68453,1485973216,VIEW,-532999578436827210,-8845298781299428018,-5430065457414428568,NonMobile
...,...,...,...,...,...,...
12169,1459345020,VIEW,7973573994178035769,-8845298781299428018,3760091107461406486,NonMobile
18783,1459285852,VIEW,6152652267138213180,-8845298781299428018,-6283148774987755959,NonMobile
692,1459274282,VIEW,-1672166631728511207,-8845298781299428018,-6283148774987755959,NonMobile
684,1459274266,VIEW,-1672166631728511207,-8845298781299428018,-6283148774987755959,NonMobile


lets take the most recent 5 articles interacted with by this user

In [17]:
most_recent_five_articles = viewed_interactions.item_id.unique()[:5].astype(str).tolist()

In [18]:
most_recent_five_articles_metadata = articles_mlfeatures[articles_mlfeatures['item_id'].isin(most_recent_five_articles)]

In [19]:
most_recent_five_articles_metadata

,creation_timestamp,item_id,lang,article_genre,training_text
2869,1481676932,8526042588044002101,en,tech,"Cloud Native enables efficient, automated infr..."
2982,1484838924,7419040071212162906,en,tech,"Google search traffic hack by targeting \""best..."
3040,1485899108,-532999578436827210,en,cloud provider news,IBM launches cloud graph database service usin...
3065,1486520793,3566197569262766169,en,cloud provider news,Google extends Gmail API for more granular ema...
3084,1486999803,-8900113512825364282,en,non tech,Report finds banks lag in customer experience ...


Lets take a look at a summary of the article text

In [20]:
for i, row in most_recent_five_articles_metadata.iterrows():
    print(row['training_text'])
    print("===================================")

Cloud Native enables efficient, automated infrastructure through modular architecture Cloud Native is an approach to build modular, automated systems that can manage complexity and accelerate development. It utilizes practices like microservices, infrastructure as code, and CI/CD. Cloud Native is a way of structuring teams, culture, and technology to utilize automation and architectures to manage complexity and unlock velocity. The real value of Cloud Native goes beyond just the technologies associated with it. Cloud Native enables more efficient and happier teams by breaking large problems into smaller pieces and reducing manual work through automation. It leads to more reliable infrastructure and applications by building in automation to handle failures. Cloud Native also enables auditable, visible, and debuggable complex applications through the tools used to build them. It allows developers to create more secure applications through least trust principles. Finally, Cloud Native ope

Lets make a helper function to automate some of the above tasks as well as allow us to use a filter

In [21]:
def return_recs_and_history(user_id, campaign_arn, article_metadata, interactions_data, filter_arn, filter_values):
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = campaign_arn,
        userId = user_id,
        numResults = 5,
        filterArn = filter_arn, #"arn:aws:personalize:us-east-1:256589169164:filter/breakingnews-genre-filter"
        filterValues = filter_values #{"GENRELIST": "\"tech\", \"cloud provider news\""}
    )
    
    recommendations_df = pd.DataFrame.from_records(get_recommendations_response['itemList'])
    
    recommendations_df = recommendations_df.merge(article_metadata, how='left', left_on='itemId', right_on='item_id')
    
    viewed_interactions = interactions_data[interactions_data['user_id'].astype(str) == user_id].sort_values('timestamp', ascending=False)
    
    most_recent_five_articles = viewed_interactions.item_id.unique()[:5].astype(str).tolist()
    
    most_recent_five_articles_metadata = article_metadata[article_metadata['item_id'].isin(most_recent_five_articles)]
    
    return recommendations_df, most_recent_five_articles_metadata

In [24]:
recommendations_df, most_recent_five_articles_metadata = return_recs_and_history(user_id = sample_user,
                                                                                campaign_arn = workshop_userpersonalization_campaign_arn,
                                                                                article_metadata = articles_mlfeatures,
                                                                                interactions_data = interaction_data,
                                                                                filter_arn = "arn:aws:personalize:us-east-1:256589169164:filter/breakingnews-genre-filter",
                                                                                filter_values = {"GENRELIST": "\"tech\", \"cloud provider news\""})